In [ ]:
pip install -U sentence-transformers

In [1]:
# Packages

#pip install -U sentence-transformers
from termcolor import colored
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
import string
import regex as re
import zipfile
import gc
from scipy.stats import boxcox
import sys 
from collections import Counter 
from tqdm import tqdm 
import pandas as pd 
import numpy as np 
import warnings, math
from termcolor import colored
import pickle
import string
from sklearn.metrics.pairwise import cosine_similarity

# for eval
from sklearn.model_selection import train_test_split
import random


# for SVD
from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix
pd.set_option('display.float_format','{:.5f}'.format)

# for EMB
from sentence_transformers import SentenceTransformer, util
import scipy
from sklearn import preprocessing 

# for TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from sklearn import preprocessing 

# for creating a network
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
EXT_FEATURES_PATH = 'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\ext_features_agg.csv'
NEW_DF_PATH = 'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_df.csv'
PROJECTS_DF_PATH = "D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\projects.csv"


ext_features = pd.read_csv(EXT_FEATURES_PATH)
df = pd.read_csv(NEW_DF_PATH)
projects_df = pd.read_csv(PROJECTS_DF_PATH)


In [3]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)

print('# donations in Train set: ', f"{len(df_train):,}")
print('# donations in Test set: ',  f"{len(df_test):,}")

print('# Donors in Train set: ', f"{len(df_train['Donor ID'].unique()):,}")
print('# Donors in Test set: ',  f"{len(df_test['Donor ID'].unique()):,}")


print('# Donors in both Train and Test sets - the ones we choose for evaluation: ',
      colored(f"{len(df_train[df_train['Donor ID'].isin(df_test['Donor ID'].values.tolist())]['Donor ID'].unique()):,}", 'blue'))



df_train = df_train[df_train['Donor ID'].isin(df_test['Donor ID'].values.tolist())].reset_index(drop=True)
df_test = df_test[df_test['Donor ID'].isin(df_train['Donor ID'].values.tolist())].reset_index(drop=True)

# sum of donation in a grouped by donor id dataset
df_main_donor_index = df.groupby(by = ['Donor ID', 'Project ID']).sum()[['Donation Amount']].reset_index().set_index('Donor ID')
df_train_donor_index = df_train.groupby(by = ['Donor ID', 'Project ID']).sum()[['Donation Amount']].reset_index().set_index('Donor ID')
df_test_donor_index = df_test.groupby(by = ['Donor ID', 'Project ID']).sum()[['Donation Amount']].reset_index().set_index('Donor ID')

# donations in Train set:  43,604
# donations in Test set:  4,845
# Donors in Train set:  790
# Donors in Test set:  617
# Donors in both Train and Test sets - the ones we choose for evaluation:  617


In [3]:
projects_df = projects_df[projects_df['Project ID'].isin(df['Project ID'])].reset_index(drop=True)

project_txt = projects_df.loc[:, 'project_txt']
projects_id = projects_df['Project ID'].tolist()


### Create Embeddings (projects)

In [6]:
# ----------------- embeddings---------------------#
model = SentenceTransformer('paraphrase-distilroberta-base-v1') # , device= 'cuda'
embeddings = model.encode(project_txt, batch_size=256, show_progress_bar=True)

In [ ]:
with open(EMBEDDING_PATH, 'wb') as f:
    pickle.dump(embeddings, f)

### Create Embeddings (donors profile)

In [8]:
EMBEDDING_PATH = 'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_project_embeddings.pickle'
project_embeddings_matrix = pd.read_pickle(EMBEDDING_PATH)

In [6]:
# ==================== Embeddings ==================== #

# ----------------------- projects profiles------------------- #
def get_project_profile_emb(project_id: str, project_embeddings_matrix):

    # get the ids
    idx = projects_id.index(project_id)
    project_profile = project_embeddings[idx:idx+1]
    
    return project_profile



def get_projects_profiles_emb(ids: pd.Series, project_embeddings_matrix):


    profiles_list = [get_project_profile_emb(project_id, project_embeddings)[0] for project_id in np.ravel([ids])]
    project_profiles = np.vstack(profiles_list)

    return project_profiles


# ----------------------- Donors profiles------------------- #

def build_donors_profile_emb(donor_id: str, df_train_donor_index: pd.DataFrame):

    # get the id of each person and the projects they
    # donated to
    donations_donor_df = df_train_donor_index.loc[donor_id]


    # get the vectors of projects this person has donated to
    donor_donated_project_intrain_profiles = get_projects_profiles_emb(donations_donor_df['Project ID'], project_embeddings)


    # get the smoothed donated amount as the weight of each project
    donor_project_strengths = np.array(donations_donor_df['Donation Amount']).reshape(-1, 1)


    # multiply the weights and tfidf vectors
    multiplication = np.multiply(donor_donated_project_intrain_profiles, donor_project_strengths)


    # now we normalize the whole vector 
    normalized_donor_preference = preprocessing.normalize(np.sum(multiplication, axis=0).reshape(1, -1))
    


    return normalized_donor_preference


def build_donors_profiles_emb(df_test_donor_index: pd.DataFrame, df_train_donor_index: pd.DataFrame, sample_size: int = None):
    
    # now for all donors we build a profile in a dictionary
    donor_profiles = {}
    donors_in_test_set = df_test_donor_index.index.unique().tolist()[:sample_size]

    for donor_id in tqdm(donors_in_test_set[:sample_size], position=0, leave=True):
        donor_profiles[donor_id] = build_donors_profile_emb(donor_id, df_train_donor_index)

    return donor_profiles

    



In [17]:

donor_profiles = build_donors_profiles_emb(df_test_donor_index, df_train_donor_index)

100%|██████████| 617/617 [00:00<00:00, 1377.75it/s]


In [ ]:
len(donor_profiles_emb['009d5fc7b87883ffad248db5150bf1fc'][0]) == 768

In [ ]:
PROFILE_EMBEDDING_PATH = 'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_donor_profiles_embeddings.pickle'

with open(PROFILE_EMBEDDING_PATH, 'wb') as f:
    pickle.dump(donor_profiles_emb, f)

### Graph

In [4]:
PROFILE_EMBEDDING_PATH = r'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_donor_profiles_embeddings.pickle'
PROJECT_EMBEDDING_PATH = r'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\Oct_28_project_embeddings.pickle'
USER_PROFILE_PATH = r'D:\Papers\Paper 3 - Recommender Systems\Recommender-systems\Files\Graph_based\user_profile.csv'

project_embeddings = pd.read_pickle(PROJECT_EMBEDDING_PATH)
donor_profiles_emb = pd.read_pickle(PROFILE_EMBEDDING_PATH)
user_profile_df = pd.read_csv(USER_PROFILE_PATH)

In [5]:
donors_id = list(donor_profiles_emb.keys())# donor_profiles_emb = user_embeddings

In [9]:
# interaction matrixes for similarity scores-------------------------------------

#project
project_interactions = linear_kernel(project_embeddings_matrix, project_embeddings_matrix)

# donors
donor_embeddings_matrix = np.array([donor_profiles_emb[id][0] for id in donors_id])
donors_interactions  = linear_kernel(donor_embeddings_matrix, donor_embeddings_matrix)


In [10]:
# DONOR - DONOR / PROJ-PROJ edges dict -----------------------------------------------------------------------


def create_edges(ids: list, interaction_matrix: np.array, top_n_similar: int = 20) -> dict:

    """
    output = {'donor_id': [(score, similar_donor_id)]}
    or 
    output = {'project_id': [(score, similar_project_id)]}

    """
    edges_dict = {}
    for idx, emb in enumerate(ids):
        similar_indices = interaction_matrix[idx].argsort()[:-top_n_similar:-1]
        
        # idx = index of selected donor/project
        # i = index of similar donor/project
        # similar_items = list(score, similar donor_id/project_id)
        similar_items = [(interaction_matrix[idx][i], ids[i]) for i in similar_indices]
        edges_dict[ids[idx]] = similar_items

    return edges_dict


donors_edges = create_edges(donors_id, donors_interactions)
project_edges = create_edges(projects_id, project_interactions)

In [11]:
# Graphs -----------------------------------------------------------

# Donor graph---------------------------------------------------------------

class Graph():
    """
    Create the donors'/projects' graph and save their information in nodes.
    """
    
    
    def __init__(self, graph_name: str):
        self.graph = {}
        self.graph_name = graph_name
    
    # function to add new nodes in the graph
    def _create_node(self, node_id: str, node_properties):
        self.graph[node_id] = node_properties 
    
    # function to view the nodes in the graph
    def _view_nodes(self):
        return self.graph
    
    # function to create edges
    def _create_edges(self, node_id, node_edges):
        if node_id in self.graph:
            self.graph[node_id]['edges'] = node_edges

In [12]:
donor_id = user_profile_df.loc[0, 'Donor ID']
next(user_profile_df[user_profile_df['Donor ID'] == donor_id].iterrows())[1]


Donor ID                                         009d5fc7b87883ffad248db5150bf1fc
subject_categories              {'Literacy & Language': 189, 'Music & The Arts...
grades                          {'Grades 9-12': 17, 'Grades PreK-2': 195, 'Gra...
metro_types                                       {'urban': 288, 'suburban': 105}
states_donated_to                                                  {'Texas': 393}
donated_projects                004a152bbe8952ea5e9d5ef89c179933,004a152bbe895...
donor_zip_first_three_digits                                                  782
donor_kmeans_cluster                                                            1
donation_amount_sum                                                     200.95590
donation_amount_median                                                    0.44020
project_cost_median                                                     591.50000
Name: 0, dtype: object

In [13]:
## initialize the donors graph
dg = Graph(graph_name = 'donor')
pg = Graph(graph_name = 'projects')


## NODES ------------------------------------------------------------------------------------------
# donors
for donor_id in donors_id:
    # donor_id = node_id
    node_properties = dict(next(user_profile_df[user_profile_df['Donor ID'] == donor_id].iterrows())[1])
    node_properties['donor_project_emb'] = donor_profiles_emb[donor_id][0]
    
    dg._create_node(donor_id, node_properties)


# projects
for idx, project_id in enumerate(projects_id):
    # donor_id = node_id
    node_properties = dict(next(projects_df[projects_df['Project ID'] == project_id].iterrows())[1])
    node_properties['project_embedding'] = project_embeddings_matrix[idx]

    del node_properties['Project Need Statement']
    #if idx==0: print(node_properties)
    pg._create_node(project_id, node_properties)



# EDGES -----------------------------------------------------------------

def create_edges_in_graph(graph: object, edges: dict, ids:list, top_n_similar: int = 5):

    for id in ids:
        node_edges = edges[id][:top_n_similar]
        graph._create_edges(id, node_edges)

create_edges_in_graph(graph = dg, edges = donors_edges, ids= donors_id, top_n_similar = 5)
create_edges_in_graph(graph = pg, edges = project_edges, ids= projects_id, top_n_similar = 5)


In [14]:
donor_id = '009d5fc7b87883ffad248db5150bf1fc'

In [15]:
dg._view_nodes()[donor_id]
#pg._view_nodes()[project_id]

{'Donor ID': '009d5fc7b87883ffad248db5150bf1fc',
 'subject_categories': "{'Literacy & Language': 189, 'Music & The Arts': 152, 'Special Needs': 43, 'Applied Learning': 40, 'Math & Science': 86, 'Health & Sports': 23, 'History & Civics': 18}",
 'grades': "{'Grades 9-12': 17, 'Grades PreK-2': 195, 'Grades 6-8': 45, 'Grades 3-5': 136}",
 'metro_types': "{'urban': 288, 'suburban': 105}",
 'states_donated_to': "{'Texas': 393}",
 'donated_projects': '004a152bbe8952ea5e9d5ef89c179933,004a152bbe8952ea5e9d5ef89c179933,004a152bbe8952ea5e9d5ef89c179933,004a152bbe8952ea5e9d5ef89c179933,004a152bbe8952ea5e9d5ef89c179933,004a152bbe8952ea5e9d5ef89c179933,00caf6fe22a68be347548718301a75e4,010732a68a6a6a7a40b6827355bd2a04,010732a68a6a6a7a40b6827355bd2a04,010732a68a6a6a7a40b6827355bd2a04,010732a68a6a6a7a40b6827355bd2a04,010732a68a6a6a7a40b6827355bd2a04,010732a68a6a6a7a40b6827355bd2a04,010732a68a6a6a7a40b6827355bd2a04,010732a68a6a6a7a40b6827355bd2a04,010732a68a6a6a7a40b6827355bd2a04,010732a68a6a6a7a40b6827